In [1]:
%gui qt5
%matplotlib qt5

from pycolleff.hhcavity import Params, HarmonicCavity
import mathphys
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.gridspec as mpl_gs
from matplotlib import rcParams, rc

rcParams.update({
    'font.size': 18, 'lines.linewidth': 2,
    'axes.grid': True, 'grid.alpha': 0.5, 'grid.linestyle': '--', 'text.usetex': True})

## A. Chao Book Example

In [ ]:
# A. Chao Book Example
hcparams = Params()
hcparams.alpha = 0.03
hcparams.E0 = 1e9
w0 = 9.4e6
T0 = 2*np.pi/w0 # s
echarge = mathphys.constants.elementary_charge
N = 1e11
hcparams.I0 = N*echarge/T0
hcparams.tunes = 0.01
hcparams.frf = 360e6
hcparams.h = 240
hcparams.Rs = 1e6
hcparams.Q = 2000
dw = 2*np.pi*(-10e3)
wrf = 2*np.pi*hcparams.frf
wr = wrf + dw
hcparams.bunlen = 1e-4
hcparams.nharm = 1

hcav = HarmonicCavity()
hcav.params = hcparams

In [ ]:
hcparams.I0*1e3

In [ ]:
growth1 = hcav.robinson_growth_rate(w=wrf, wr=wr, approx=False)
growth2 = hcav.robinson_growth_rate(w=wrf, wr=wr, approx=True)

1/growth1, 1/growth2

## MAX-IV Comparison

In [ ]:
hcparams = Params()
hcav = HarmonicCavity()

# hcparams.sirius_params()
# # hcparams.Q = 21600
# hcparams.Q = int(2e8)
# hcparams.Rs = 88.4 * hcparams.Q
# hcparams.I0 = 350e-3
hcparams.maxiv_params()
# hcparams.Rs = 2.017e6
hcparams.Rs = 4.2e6
hcav.params = hcparams

hcav.harmonic_phase = hcav.phih_harmonic_flat_potential
hcav.psih_harmonic = hcav.psih_harmonic_flat_potential
hcav.detune_angle = hcav.psih_harmonic

hcav.print_flat_potential()

### Equilibrium Distribution

In [ ]:
dt = 0.9
frf = hcav.params.frf
c = mathphys.constants.light_speed
npts = int(1e6)
tau = np.linspace(-1/frf/2 * dt, 1/frf/2 * dt,  npts)
z = tau * c
wrf = hcav.params.wrf
phase = wrf*z/c

In [ ]:
vmain, vmain_pert, vharm = hcav.calc_voltages(z)
detune = hcav.detune_passive_cavity(hcav.params.Rs)
# hcav.detune_angle = detune
# detune = -96.558 * np.pi/180
hcav.detune_angle = detune

# hcav.wr = 3*hcav.params.wrf + 60.36*1e3 * 2*np.pi
# detune = hcav.detune_angle
vharm = hcav.calc_passive_voltage(z, Rs=hcav.params.Rs, detune_phase=detune)
fig = hcav.plot_voltages(z, vmain, vharm, vmain_pert+vharm)

In [ ]:
# rho0 = hcav.calc_distribution(z, harmonic=False)
# rhohc = hcav.calc_distribution(z, harmonic=True)

rho0 = hcav.calc_distribution(z, voltage=vmain)
rhohc = hcav.calc_distribution(z, voltage=vmain_pert + vharm)

### Scalar self-consistency

In [ ]:
# Loop Form Factor
rho = rhohc
wrf = hcav.params.wrf
fhist = []
ghist = []
tol = 1e-8
for i in range(100):
    fhist.append(hcav.form_factor)
    vmain_pert, vharm, rhof = loop_form_factor(hcav, rho, include_phase=False, update_detune=True)
    fnew = hcav.complex_form_factor(z, 3*wrf, rhof)
    ghist.append(abs(fhist[-1]) - abs(fnew))
    rho = rhof
    print(i, hcav.form_factor, ghist[-1], hcav.detune_angle)
    if abs(ghist[-1]) < tol:
        break

In [ ]:
maxiv1 = rho

### Full self-consistency

In [ ]:
# Loop Form Factor
rho = rhohc
wrf = hcav.params.wrf
fhist = []
ghist = []
tol = 1e-8
for i in range(100):
    fhist.append(hcav.form_factor)
    vmain_pert, vharm, rhof = loop_form_factor(hcav, rho, include_phase=True, update_detune=True)
    fnew = hcav.complex_form_factor(z, 3*wrf, rhof)
    ghist.append(abs(fhist[-1]) - abs(fnew))
    rho = rhof
    print(i, hcav.form_factor, ghist[-1], hcav.detune_angle)
    if abs(ghist[-1]) < tol:
        break

In [ ]:
maxiv2 = rho

In [ ]:
z0 = hcav.calc_sync_phase(z, rho0)
z0hc = hcav.calc_sync_phase(z, rho)

plt.plot((z-z0)*100, rho0*hcav.params.Ib, label='Without 3HC')
# plt.plot((z-z0)*100, rhohc0, label='With 3HC - Flat Potential')
plt.plot((z-z0hc)*100, rho*hcav.params.Ib, '--', label='With 3HC - Passive Cavity')
plt.xlabel('z [cm]')
plt.ylabel('Normalized bunch distribution [a.u.]')
# plt.title('Cold parking +500kHz')
plt.legend()
plt.grid(True, alpha=0.5, ls='--')
plt.show()

In [ ]:
maxdist = np.loadtxt('/home/murilo/Downloads/maxiv_distribution_best.txt')

In [ ]:
ph = maxdist[:, 0]
val1 = maxdist[:, 1]
val2 = maxdist[:, 2]

fig = plt.figure(figsize=(8, 4))
gs = mpl_gs.GridSpec(1, 2)
ax1 = plt.subplot(gs[0, 0])
ax2 = plt.subplot(gs[0, 1])

ax1.plot(ph, val1 , '-', label='Paper')
# conv1 = np.max(val1)/np.max(maxiv1)
lambrf = 2*np.pi/(wrf/c)
conv1 = hcav.params.I0*lambrf
filt = np.logical_and(phase > np.min(ph), phase < np.max(ph))
ax1.plot(phase[filt], maxiv1[filt]*conv1, '--', lw=2, label='Simul.')
ax1.legend()
ax1.grid(True, ls='--', alpha=0.5)
ax1.set_xlabel('phase [rad]')
ax1.set_ylabel('current distribution [A]')
ax1.set_title('Scalar self-consistency')
bun1 = hcav.calc_bunch_length(z, maxiv1)*1e3
textstr = r'$\sigma_z$ = {:.2f} mm'.format(bun1)
textstr += '\n'
textstr += r'$I_p$ = {:.2f} A'.format(np.max(maxiv1*conv1))
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
ax1.text(0.05, 0.95, textstr, transform=ax1.transAxes, fontsize=12,
        verticalalignment='top', bbox=props)


ax2.plot(ph, val2, '-', label='Paper')
# conv2 = np.max(val2)/np.max(maxiv2)
conv2 = hcav.params.I0*lambrf
ax2.plot(phase[filt], maxiv2[filt]*conv2, '--', lw=2, label='Simul.')
ax2.grid(True, ls='--', alpha=0.5)
ax2.set_xlabel('phase [rad]')
ax2.set_ylabel('current distribution [A]')
ax2.set_title('Full self-consistency')

bun2 = hcav.calc_bunch_length(z, maxiv2)*1e3
textstr = r'$\sigma_z$ = {:.2f} mm'.format(bun2)
textstr += '\n'
textstr += r'$I_p$ = {:.2f} A'.format(np.max(maxiv2*conv2))
ax2.text(0.05, 0.95, textstr, transform=ax2.transAxes, fontsize=12,
        verticalalignment='top', bbox=props)
# ax2.legend()
# plt.plot(max_dist_complex[:, 0], max_dist_complex[:, 1], '--')
plt.tight_layout(True)
plt.show()
# plt.savefig('maxiv_comparison_paper.png', dpi=300, format='png')

In [ ]:
pot0 = hcav.integrated_potential(z, voltage=vmain)
# pothc0 = hcav.integrated_potential(z, voltage=vmain_pert0+vharm0)
pothc = hcav.integrated_potential(z, voltage=vmain_pert+vharm)


norm = (hcav.params.alpha * hcav.params.espread)**2
plt.plot(z*100, pot0/norm, label='Without 3HC')
# plt.plot((z-z0)*100, pothc0*1e10, label='With 3HC - Flat Potential')
plt.plot((z-z0)*100, pothc/norm, '--', label='With 3HC - Passive Cavity')


plt.xlabel('z [cm]')
plt.ylabel(r'Integrated RF potential $\Phi$ [a.u.]')
# plt.title('Cold parking +500kHz')
plt.legend()
plt.grid(True, alpha=0.5, ls='--')
plt.show()

In [ ]:
hcav.wr = 3*hcav.params.wrf + 2*np.pi*(500e3)
detune = hcav.detune_angle
hcav.harmonic_phase = (hcav.detune_angle + np.pi/2)/hcav.params.nharm
vharm = hcav.calc_passive_voltage(z, Rs=hcav.params.Rs, detune_phase=detune)

#### Bunch Length

In [ ]:
sig1 = hcav.calc_bunch_length(z, rho0)*1e3
sig2 = hcav.calc_bunch_length(z, rho)*1e3
sig1, sig2, sig2/sig1

In [ ]:
# detune = hcav.detune_passive_cavity(hcav.params.Rs)
# hcav.detune_angle = detune
detune = hcav.detune_angle
hcav.harmonic_phase = (hcav.detune_angle + np.pi/2)/hcav.params.nharm

In [ ]:
# detune = hcav.psih_harmonic
vpass = hcav.calc_passive_voltage(z, hcav.params.Rs, detune)


vmain, vmain_pert, vharm = hcav.calc_voltages(z)
fig = hcav.plot_voltages(z, vmain, vmain_pert, vpass)

In [ ]:
angle = hcav.psih_harmonic
wr = hcav.wr
wrf = hcav.params.wrf

In [ ]:
df = (wr - 3*wrf)/2/np.pi
print(df/1e3)

### Growth Rates

In [ ]:
growth1 = hcav.robinson_growth_rate(w=3*wrf, wr=wr, approx=False)
growth2 = hcav.robinson_growth_rate(w=3*wrf, wr=wr, approx=True)

growth1, growth2

In [ ]:
npts = int(1e6)
w0 = wrf/hcav.params.h
# w = np.linspace(-(wr+3*wrf), wr+3*wrf, npts)
w = np.linspace(wr-wrf, wr + wrf, npts)
w = np.r_[-np.flipud(w), w]
deltaw, Zl, wp, interpol_Z, spectrum = hcav.tuneshifts_cbi(w=w, wr=wr, m=1, nbun_fill=hcav.params.h, radiation=True)

In [ ]:
deltaw.imag[0]

In [ ]:
plt.plot(deltaw.imag, '.-')
plt.xlabel('mode index')
plt.ylabel(r'$\tau^{-1}$ for $m=1$ [$s^{-1}$]')
plt.title('Longitudinal growth rate for passive 3HC cold parking')
plt.grid(True, alpha=0.5, ls='--')

In [ ]:
plt.semilogy(w, Zl.real/1e6, '.')
idx = 0
plt.semilogy(wp[idx, :], interpol_Z[idx, :].real/1e6, 'o')

plt.xlabel(r'$\omega$ [rad/s]')
plt.ylabel(r'Re$(Z_L)$ [M$\Omega$] ')
plt.title('Mode {:d}'.format(idx))

plt.grid(True, alpha=0.5, ls='--')

In [ ]:
plt.semilogy(w, Zl.real/1e6, '-')

idx1 = 0
plt.semilogy(wp[idx1, :], interpol_Z[idx1, :].real/1e6, 'o', label=f'Mode {idx1:d}')
idx2 = 1
plt.semilogy(wp[idx2, :], interpol_Z[idx2, :].real/1e6, '+', label=f'Mode {idx2:d}')
idx3 = 2
plt.semilogy(wp[idx3, :], interpol_Z[idx3, :].real/1e6, 'd', label=f'Mode {idx3:d}')
# idx4 = 3
# plt.semilogy(wp[idx4, :], interpol_Z[idx4, :].real/1e6, 'x', label=f'Mode {idx4:d}')

plt.legend()
plt.xlabel(r'$\omega$ [rad/s]')
plt.ylabel(r'Re$(Z_L)$ [M$\Omega$] ')
plt.title(r'$R_s=$ {:.3f}$M\Omega$, $Q=$ {:.1e}, Sirius'.format(
    hcav.params.Rs/1e6, hcav.params.Q))

plt.grid(True, alpha=0.5, ls='--')

## Sirius Studies

In [39]:
hcparams = Params()
hcav = HarmonicCavity()

hcparams.sirius_params()
# hcparams.Q = 21600
# hcparams.Q = 4e8
# hcparams.Rs = 90 * hcparams.Q

# hcparams.Q = 4e8
# hcparams.Rs = 180 * hcparams.Q

# hcparams.Q = 4e8
hcparams.Rs = 11876921.43677626
hcparams.Q = hcparams.Rs/90

# hcparams.I0 = 350e-3
# hcparams.maxiv_params()
# hcparams.Rs = 2.017e6
# hcparams.Rs = 4.2e6
hcav.params = hcparams

hcav.harmonic_phase = hcav.phih_harmonic_flat_potential
hcav.psih_harmonic = hcav.psih_harmonic_flat_potential
hcav.detune_angle = hcav.psih_harmonic
wr = hcav.wr

hcav.print_flat_potential()

k flat potential              : +000.317 
harmonic phase [deg]          : -002.190 
harmonic tuning angle [deg]   : -096.571 
detuning frequency    [kHz]   : +049.305 
shunt impedance flat [M.ohm]  : +11.8769 
unperturbed sync. phase [deg] : +163.122 
perturbed sync. phase [deg]   : +160.936 



In [38]:
hcav.shunt_impedance

11876921.43677626

### Equilibrium Distribution

In [40]:
dt = 0.5
frf = hcav.params.frf
c = mathphys.constants.light_speed
npts = int(1e3)+1
tau = np.linspace(-1/frf/2 * dt, 1/frf/2 * dt,  npts)
z = tau * c
wrf = hcav.params.wrf
phase = wrf*z/c

vmain, vmain_pert, vharm = hcav.calc_voltages(z)
detune = hcav.detune_passive_cavity(hcav.params.Rs)
hcav.detune_angle = detune
# detune = -96.558 * np.pi/180
# hcav.detune_angle = detune
# hcav.wr = 3*hcav.params.wrf + 60.36*1e3 * 2*np.pi
# detune = hcav.detune_angle

Vrf = hcav.params.Vrf
wrf = 2*np.pi*hcav.params.frf
phis0 = hcav.params.sync_phase
phis_pert = hcav.perturbed_sync_phase
kh = hcav.k_harmonic_flat_potential
nh = hcav.params.nharm
phih = hcav.harmonic_phase

phase = wrf*z/c
vmain = Vrf*np.sin(phase + phis0)
vmain_pert = Vrf*np.sin(phase + phis_pert)
# vharm = Vrf*kh*np.sin(nh*phase + nh*phih)
# vtotal = vmain_pert + vharm

vharm = hcav.calc_passive_voltage(z, Rs=hcav.params.Rs, detune_phase=detune)
vtotal = vmain_pert + vharm
fig = hcav.plot_voltages(z, vmain, vharm, vtotal)

# rho0 = hcav.calc_distribution(z, harmonic=False)
# rhohc = hcav.calc_distribution(z, harmonic=True)
rho0 = hcav.calc_distribution(z, voltage=vmain)
rhohc = hcav.calc_distribution(z, voltage=vtotal)
# fig.savefig('sirius_voltage_3hc.png', dpi=600, format='png')

In [23]:
fig = plt.figure(figsize=(8, 6))
gs = mpl_gs.GridSpec(1, 1)
ax1 = plt.subplot(gs[0, 0])

ax1.plot(z*100, vmain * 1e-6, label=r'$V_{rf}$ Main', color='C0')
ax1.plot(z*100, vharm * 1e-6, label=r'$V_{3h}$ 3HC', color='C1')
ax1.plot(
    z*100, vtotal * 1e-6, label=r'$V_{rf} + V_{3h}$ Total', color='C3')
ax1.axhline(
    y=hcav.params.U0*1e-6, color='tab:gray', ls='--', label=r'$U_0$')

ax1.set_xlabel(r'$z$ [cm]')
ax1.set_ylabel('RF voltage [MV]')
ax1.legend(loc='upper right')
ax1.grid(ls='--', alpha=0.5)

In [42]:
plt.figure()
pot0 = hcav.integrated_potential(z, voltage=vmain)
# pothc0 = hcav.integrated_potential(z, voltage=vmain_pert0+vharm0)
pothc = hcav.integrated_potential(z, voltage=vtotal)

norm = (hcav.params.alpha * hcav.params.espread)**2
plt.plot(z*100, pot0/norm, label='Without 3HC', lw=2)
# plt.plot((z-z0)*100, pothc0*1e10, label='With 3HC - Flat Potential')
plt.plot(z*100, pothc/norm, '-', label='With 3HC', lw=2)

plt.xlabel('z [cm]')
plt.ylabel(r'RF potential $\Phi$ [a.u.]')
# plt.title('Cold parking +500kHz')
plt.legend()
plt.grid(True, alpha=0.5, ls='--')
# plt.savefig('sirius_potential_3hc.png', dpi=600, format='png')
plt.show()

In [44]:
lambrf = 2*np.pi/(wrf/c)
conv = hcav.params.I0*lambrf

plt.figure()
plt.plot(z*100, rho0*conv, label='Without 3HC', lw=2)
# plt.plot((z-z0)*100, pothc0*1e10, label='With 3HC - Flat Potential')
plt.plot(z*100, rhohc*conv, '-', label='With 3HC', lw=2)

plt.xlabel('z [cm]')
plt.ylabel(r'Longitudinal distribution [A]')
# plt.title('Cold parking +500kHz')
plt.xlim([-7.5, 7.5])
plt.legend()
plt.grid(True, alpha=0.5, ls='--')
# plt.savefig('sirius_distribution_3hc.png', dpi=600, format='png')
plt.show()

In [ ]:
np.trapz(rho0**2, z)/np.trapz(rhohc**2, z)

### Scalar self-consistency

In [ ]:
# Loop Form Factor
rho = rhohc
wrf = hcav.params.wrf
fhist = []
ghist = []
tol = 1e-12
for i in range(100):
    fhist.append(hcav.form_factor)
    vmain_pert, vharm, rhof = loop_form_factor(hcav, rho, include_phase=False, update_detune=True)
    fnew = hcav.complex_form_factor(z, 3*wrf, rhof)
    ghist.append(abs(fhist[-1]) - abs(fnew))
    rho = rhof
    print(i, hcav.form_factor, ghist[-1], hcav.detune_angle)
    if abs(ghist[-1]) < tol:
        break

In [ ]:
sirius1 = rho
dtune1 = hcav.wr

### Complex self-consistency

In [45]:
hcav.form_factor = 1
psi_opt = hcav.detune_passive_cavity(Rs=hcav.params.Rs)
hcav.detune_angle = psi_opt
dw = hcav.wr - 3*hcav.params.wrf
df = dw/2/np.pi

### Fixed K

In [ ]:
dfreqs[np.argmin(peak_curr)]/1e3
best_freq = 42.248525706516496*1e3

In [ ]:
# vari = 1e-3/100
# detunes = np.linspace(psi_opt*(1-3*vari), psi_opt*(1+vari), 11)
# detunes = [psi_opt, psi_opt*(1-1e-5/100)]
peak_curr = []
bun_lens = []
# detunes = [4.712307380481071]
# dfreqs = np.linspace(40, 55, 11)*1e3

hcav.form_factor = 1
psi_opt = hcav.detune_passive_cavity(Rs=hcav.params.Rs)
hcav.detune_angle = psi_opt
dw = hcav.wr - 3*hcav.params.wrf
# df_opt = dw/2/np.pi
# df_opt = 45575.687342148754

rho0 = rhohc
wrf = hcav.params.wrf

lambrf = 2*np.pi/(wrf/c)
conv = hcav.params.I0*lambrf
# dfreqs = np.linspace(41, 45, 101)*1e3
# dtunes = np.linspace(-5, +5, 25)*1e3
# dfreqs = df_opt + dtunes
# dfreqs = np.array([df_opt])
dists = []
errs = []
entropys = []

# dfreqs = np.array([best_freq])
# dfreqs = np.array([df_opt])
# nbeta = 100
nbeta = 0
for idx, dfreq in enumerate(dfreqs):
    # Loop Form Factor
    rho = rhohc
    hcav.wr = 3*wrf + 2*np.pi*dfreq
    rhof, err, entropy, nit = hcav.convergence_distribution(
        z=z, rho0=rho0, niter=100, tol=1e-10, beta=nbeta, 
        method='anderson_acc', update_detune=True, include_phase=True, print_iter=False)
    peak_curr.append(np.max(rhof)*conv)
    bun_lens.append(hcav.calc_bunch_length(z, rhof)*1e3)
    dists.append(rhof)
    errs.append(err)
    entropys.append(entropy)
    print('{:03d}/{:03d} - err: {:.1e} - beta: {:06.2f} - niter: {:05d}'.format(idx+1, dfreqs.size, err[-1], nbeta, nit))
    nbeta /= 1.05

In [ ]:
(hcav.wr-3*wrf)/2/np.pi, df_opt

In [ ]:
f = plt.figure()
ax = plt.axes()
jet = plt.get_cmap('jet') 
cNorm  = colors.Normalize(vmin=dtunes.min(), vmax=dtunes.max())
scalarMap = cm.ScalarMappable(norm=cNorm, cmap=jet)

# perc_diff = np.linspace(0.9, 1.1, 51)
cNorm  = colors.Normalize(vmin=(df_opt+dtunes).min()/1e3, vmax=(df_opt+dtunes).max()/1e3)
scalarMapdiff = cm.ScalarMappable(norm=cNorm, cmap=jet)
ax.set_xlim([-10, 5])
ax.set_xlabel(r'$z$ [cm]')
ax.set_ylabel('Distribution [a.u.]')
plt.grid(True, ls='--', alpha=0.5, color='gray')
plt.colorbar(scalarMapdiff, label=r'Frequency detune [kHz]')
plt.tight_layout(True)

for idx, lamb in enumerate(dists):
    clr = scalarMap.to_rgba(dtunes[idx])
    ax.plot(-z*100, lamb, color=clr)
    plt.pause(0.01)
#     ax.clear()
# textstr =r'$\sigma_z$ = {:.2f} mm'.format(bun_lens[0])
# textstr += '\n'
# textstr += r'$I_p$ = {:.2f} A'.format(peak_curr[0])
# props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
# ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=12,
#         verticalalignment='top', bbox=props)

# plt.title('Distribution with minimum peak current')
plt.tight_layout(True)
# f.savefig('dist_fp_condition.png', dpi=300)
# plt.show()

In [ ]:
bun_lens, peak_curr

### Fixed K and detune

In [46]:
# vari = 1e-3/100
# detunes = np.linspace(psi_opt*(1-3*vari), psi_opt*(1+vari), 11)
# detunes = [psi_opt, psi_opt*(1-1e-5/100)]
peak_curr = []
bun_lens = []
# detunes = [4.712307380481071]
# dfreqs = np.linspace(40, 55, 11)*1e3

hcav.form_factor = 1
psi_opt = hcav.detune_passive_cavity(Rs=hcav.params.Rs)
hcav.detune_angle = psi_opt
dw = hcav.wr - 3*hcav.params.wrf
df = dw/2/np.pi

dfreqs = np.array([df])
wrs = 3*hcav.params.wrf + 2*np.pi*dfreqs
rho0 = rhohc
wrf = hcav.params.wrf

lambrf = 2*np.pi/(wrf/c)
conv = hcav.params.I0*lambrf
factor = 1
# perc_diff = np.linspace(0.95, 1.05, 51*factor)
# perc_diff = np.array([1, 1.01, 1.05])
# khs = hcav.k_harmonic_flat_potential * perc_diff
khs = np.array([hcav.k_harmonic_flat_potential])
dists = []
errs = []
entropys = []

nbeta = 1
# beta_min = 1
# beta_max = 10
# nbeta = np.linspace(beta_min, beta_max, khs.size)
for idx, k in enumerate(khs):
    # Loop Form Factor
    rho = rhohc
    rhof, err, entropy, nit = hcav.convergence_distribution(
        z=z, rho0=rho0, niter=1000, tol=1e-10, beta=nbeta, 
        method='anderson_acc', kh=k, update_detune=True, include_phase=True, print_iter=False)
    peak_curr.append(np.max(rhof)*conv)
    bun_lens.append(hcav.calc_bunch_length(z, rhof)*1e3)
    dists.append(rhof)
    errs.append(err)
    entropys.append(entropy)
#     nbeta *= 1.05/factor
    print(perc_diff[idx])
    print('{:03d}/{:03d} - err: {:.1e} - beta: {:06.2f} - niter: {:05d}'.format(idx+1, khs.size, err[-1], nbeta, nit))

/home/facs/repos/collective_effects/pycolleff/pycolleff/hhcavity.py:440: RuntimeWarning: divide by zero encountered in log
  sint = rho*_np.log(rho)
/home/facs/repos/collective_effects/pycolleff/pycolleff/hhcavity.py:440: RuntimeWarning: invalid value encountered in multiply
  sint = rho*_np.log(rho)


0.95
001/001 - err: 9.7e-11 - beta: 001.00 - niter: 00442


### Plots

In [47]:
dw = hcav.wr - 3*hcav.params.wrf
df = dw/2/np.pi
print(df)

45942.96523777304


In [49]:
f = plt.figure()
ax = plt.axes()
jet = plt.get_cmap('jet') 
cNorm  = colors.Normalize(vmin=khs[0], vmax=khs[-1])
scalarMap = cm.ScalarMappable(norm=cNorm, cmap=jet)

# perc_diff = np.linspace(0.9, 1.1, 51)
diff = (1-perc_diff)*100
cNorm  = colors.Normalize(vmin=diff[0], vmax=diff[-1])
scalarMapdiff = cm.ScalarMappable(norm=cNorm, cmap=jet)
# ax.set_xlim([-10, 5])
ax.set_xlabel(r'$z$ [cm]')
ax.set_ylabel('Distribution [a.u.]')
plt.grid(True, ls='--', alpha=0.5, color='gray')
plt.colorbar(scalarMapdiff, label=r'Diff. to FP voltage [\%]')
plt.tight_layout(True)

for idx, lamb in enumerate(dists):
    clr = scalarMap.to_rgba(khs[idx])
    ax.plot(z*100, lamb, color=clr)
    plt.pause(0.2)
#     ax.clear()


# f.savefig('scan_hvoltage_dist.png', dpi=300)
plt.show()

In [11]:
hcav.params.Rs/hcav.params.Q, hcav.params.Q/1e8

(180.0, 4.0)

In [ ]:
f = plt.figure()
jet = plt.get_cmap('jet') 
cNorm  = colors.Normalize(vmin=khs[0], vmax=khs[-1])
scalarMap = cm.ScalarMappable(norm=cNorm, cmap=jet)
for idx, err in enumerate(errs):
    clr = scalarMap.to_rgba(khs[idx])
    plt.plot(err, color=clr)
plt.yscale('log')
plt.show()

In [ ]:
f = plt.figure()
jet = plt.get_cmap('jet') 
cNorm  = colors.Normalize(vmin=khs[0], vmax=khs[-1])
scalarMap = cm.ScalarMappable(norm=cNorm, cmap=jet)
off = 0
for idx, etpy in enumerate(entropys):
    clr = scalarMap.to_rgba(khs[idx])
    plt.plot(etpy+off, '-', color=clr)
    off += 0.5
plt.yscale('log')
plt.show()

In [ ]:
f = plt.figure()
plt.plot(khs, peak_curr, '-', label='Peak Current')
plt.plot(khs, bun_lens, '-', label='Bunch Length')
plt.vlines(hcav.k_harmonic_flat_potential, ymin=5, ymax=25, ls='-')
plt.vlines(khs[np.argmin(peak_curr)], ymin=5, ymax=25, ls='--')
plt.grid(True, ls='--', alpha=0.5, color='gray')
plt.legend()
plt.show()

In [ ]:
khs[np.argmin(peak_curr)]

In [ ]:
plt.plot(dfreqs/1e3, peak_curr, 'o-', label='Peak Current')
plt.plot(dfreqs/1e3, bun_lens, 'o-', label='Bunch Length')
plt.legend()
plt.show()

In [ ]:
fig, ax1 = plt.subplots()

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

ax1.plot(dfreqs/1e3, bun_lens, '-', color='C0')
ax2.plot(dfreqs/1e3, peak_curr, '-', color='C1')

ax1.vlines(df_opt/1e3, ymin=np.min(bun_lens), ymax=np.max(bun_lens), label=r'Detune FP - $\Delta f$ = {:.3f}kHz'.format(df_opt/1e3))
# ax1.vlines(dfreqs[np.argmax(bun_lens)]/1e3, ymin=np.min(bun_lens), ymax=np.max(peak_curr), ls='--', label=r'Max. $\sigma_z$ - $\Delta f$ = {:.3f}kHz'.format(dfreqs[np.argmax(bun_lens)]/1e3))
ax1.vlines(dfreqs[np.argmin(peak_curr)]/1e3, ymin=np.min(bun_lens), ymax=np.max(bun_lens), ls=':', label=r'Min. $I_p$ - $\Delta f$ = {:.3f}kHz'.format(dfreqs[np.argmin(peak_curr)]/1e3))
ax1.set_xlabel('$\Delta f$ [kHz]')
ax1.set_ylabel('Bunch Length [mm]', color='C0')
ax2.set_ylabel('Peak Current [A]', color='C1')

ax1.grid(True, ls='--', alpha=0.5)
ax2.grid(True, ls='--', alpha=0.5)
ax1.legend(loc='upper center')

ax1.tick_params(axis='y', labelcolor='C0')
ax2.tick_params(axis='y', labelcolor='C1')
fig.tight_layout()
# fig.savefig('detune_scan_bun_peak_fixed_k.png', dpi=600, format='png')

In [ ]:
fig, ax1 = plt.subplots()

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

ax1.plot(khs, bun_lens, '-', color='C0')
ax2.plot(khs, peak_curr, '-', color='C1')

k_opt = hcav.k_harmonic_flat_potential
ax1.vlines(k_opt, ymin=np.min(bun_lens), ymax=np.max(bun_lens), label=r'FP - $k$ = {:.3f}'.format(k_opt))
# ax1.vlines(dfreqs[np.argmax(bun_lens)]/1e3, ymin=np.min(bun_lens), ymax=np.max(peak_curr), ls='--', label=r'Max. $\sigma_z$ - $\Delta f$ = {:.3f}kHz'.format(dfreqs[np.argmax(bun_lens)]/1e3))
ax1.vlines(khs[np.argmin(peak_curr)], ymin=np.min(bun_lens), ymax=np.max(bun_lens), ls=':', label=r'Min. $I_p$ - $k$ = {:.3f}'.format(khs[np.argmin(peak_curr)]))
ax1.set_xlabel('$k$ harmonic')
ax1.set_ylabel('Bunch Length [mm]', color='C0')
ax2.set_ylabel('Peak Current [A]', color='C1')

ax1.grid(True, ls='--', alpha=0.5)
ax2.grid(True, ls='--', alpha=0.5)
ax1.legend(loc='upper center')

ax1.tick_params(axis='y', labelcolor='C0')
ax2.tick_params(axis='y', labelcolor='C1')
fig.tight_layout()
# fig.savefig('detune_scan_bun_peak_fixed_k.png', dpi=600, format='png')

In [ ]:
pot0 = hcav.integrated_potential(z, voltage=vmain)
# pothc0 = hcav.integrated_potential(z, voltage=vmain_pert0+vharm0)
vmain_pert, vharm, rho = hcav.loop_form_factor(
                z, rho, include_phase=True, update_detune=False)
pothc = hcav.integrated_potential(z, voltage=vmain_pert+vharm)

norm = (hcav.params.alpha * hcav.params.espread)**2
plt.plot(z*100, pot0/norm, label='Without 3HC')
# plt.plot((z-z0)*100, pothc0*1e10, label='With 3HC - Flat Potential')
plt.plot(z*100, pothc/norm, '--', label='With 3HC - Passive Cavity')

plt.xlabel('z [cm]')
plt.ylabel(r'Integrated RF potential $\Phi$ [a.u.]')
# plt.title('Cold parking +500kHz')
plt.legend()
plt.grid(True, alpha=0.5, ls='--')
plt.show()

### Bunch-Length

In [ ]:
sig1 = hcav.calc_bunch_length(z, rho0)*1e3
sig2 = hcav.calc_bunch_length(z, rho)*1e3
sig1, sig2, sig2/sig1

In [ ]:
# detune = hcav.detune_passive_cavity(hcav.params.Rs)
# hcav.detune_angle = detune
detune = hcav.detune_angle
hcav.harmonic_phase = (hcav.detune_angle + np.pi/2)/hcav.params.nharm

# detune = hcav.psih_harmonic
vpass = hcav.calc_passive_voltage(z, hcav.params.Rs, detune)
vmain, vmain_pert, vharm = hcav.calc_voltages(z)
fig = hcav.plot_voltages(z, vmain, vmain_pert, vpass)

In [ ]:
# angle = hcav.psih_harmonic
angle = hcav.detune_angle
wr = hcav.wr
wrf = hcav.params.wrf

df = (wr - 3*wrf)/2/np.pi
print(df/1e3)

In [ ]:
angle*180/np.pi-360

### Growth Rates

In [ ]:
# wr = 3*wrf + dfreqs[np.argmin(peak_curr)]*2*np.pi
# wr = 3*wrf + df_opt*2*np.pi
growth1 = hcav.robinson_growth_rate(w=3*wrf, wr=wr, approx=False)
growth2 = hcav.robinson_growth_rate(w=3*wrf, wr=wr, approx=True)

growth1, growth2

In [ ]:
1/growth1

In [ ]:
npts = int(1e6)
w0 = wrf/hcav.params.h
# w = np.linspace(-(wr+3*wrf), wr+3*wrf, npts)
w = np.linspace(wr-wrf, wr + wrf, npts)
w = np.r_[-np.flipud(w), w]
deltaw, Zl, wp, interpol_Z, spectrum = hcav.tuneshifts_cbi(w=w, wr=wr, m=1, nbun_fill=hcav.params.h, radiation=True)

In [ ]:
deltaw

In [ ]:
plt.plot(deltaw.imag, '.-')
plt.xlabel('mode index')
plt.ylabel(r'$\tau^{-1}$ for $m=1$ [$s^{-1}$]')
plt.title('Longitudinal growth rate for maximum bunch length')
plt.grid(True, alpha=0.5, ls='--')
# plt.savefig('growth_rate_max_bun_len.png', dpi=600, format='png')

In [ ]:
plt.semilogy(w, Zl.real/1e6, '-')
idx = 0
plt.semilogy(wp[idx, :], interpol_Z[idx, :].real/1e6, 'o')

plt.xlabel(r'$\omega$ [rad/s]')
plt.ylabel(r'Re$(Z_L)$ [M$\Omega$] ')
plt.title('Mode {:d}'.format(idx))

plt.grid(True, alpha=0.5, ls='--')

In [ ]:
plt.semilogy(w, Zl.real/1e6, '-')

idx1 = 0
plt.semilogy(wp[idx1, :], interpol_Z[idx1, :].real/1e6, 'o', label=f'Mode {idx1:d}')
idx2 = 1
plt.semilogy(wp[idx2, :], interpol_Z[idx2, :].real/1e6, '+', label=f'Mode {idx2:d}')
idx3 = 2
plt.semilogy(wp[idx3, :], interpol_Z[idx3, :].real/1e6, 'd', label=f'Mode {idx3:d}')
# idx4 = 3
# plt.semilogy(wp[idx4, :], interpol_Z[idx4, :].real/1e6, 'x', label=f'Mode {idx4:d}')

plt.legend()
plt.xlabel(r'$\omega$ [rad/s]')
plt.ylabel(r'Re$(Z_L)$ [M$\Omega$] ')
plt.title(r'$R_s=$ {:.3f}$M\Omega$, $Q=$ {:.1e}, Sirius'.format(
    hcav.params.Rs/1e6, hcav.params.Q))

plt.grid(True, alpha=0.5, ls='--')